In [1]:
import json
import re
from mstr_robotics.prepare_AI_data import parse_json
from mstr_robotics.mstr_classes import mstr_global
from mstr_robotics.json_checksum_handler import json_checksum_handler
from mstr_robotics.read_out_prj_obj import read_gen
from mstr_robotics.redis_db import  redis_bi_analysis
from mstr_robotics._helper import msic
from mstr_robotics._connectors import mstr_api
from mstr_robotics.prepare_AI_data import mstr_to_json,redis_mstr_json
import pandas as pd

from mstrio.connection import Connection
i_mstr_global=mstr_global()
i_msic=msic()
i_read_gen=read_gen()
i_parse_json=parse_json()
i_json_checksum_handler=json_checksum_handler()
i_mstr_to_json=mstr_to_json()
i_mstr_api=mstr_api()
i_redis_mstr_json=redis_mstr_json()


project_id="B7CA92F04B9FAE8D941C3E9B7E0CD754"

def read_out_project_all(conn):
    

    obj_types_d={"FILTER": 1,"REPORT_DEFINITION": 3,"METRIC": 4,"BASE_FORMULAR":7,"PROMPT": 10,"ATTRIBUTE": 12,"FACT": 13,
            "DIMENSION": 14,"TABLE": 15,"DATAMART_REPORT": 16,"DBLOGIN": 30,"DBCONNECTION": 31,"PROJECT": 32,"USER": 34,
            "USERGROUP": 34,"ROLE": 43,"CONSOLIDATION": 47,"CONSOLIDATION_ELEMENT": 48,"DBTABLE": 53,"DOCUMENT_DEFINITION": 55,
            "DRILL_MAP": 56,"SECURITY_FILTER": 58}
    #obj_types_d={"DOCUMENT_DEFINITION": 55}
    all_obj_d_l=[]
    for obj_type in obj_types_d:
        obj_d_l=search_obj_by_type(conn, obj_type_id=obj_types_d[obj_type])
        for obj in obj_d_l:
            obj_d={"id":obj["id"],"type":obj["type"],"subtype":obj["subtype"],"name":obj["name"]}
            all_obj_d_l.append(obj_d.copy())
    return all_obj_d_l

def search_obj_by_type(conn, obj_type_id):
    url = f'{conn.base_url}/api/metadataSearches/results?pattern=4&domain=2&scope=rooted&type={obj_type_id}&visibility=ALL'
    search_inst_resp = conn.post(url)
    
    if search_inst_resp.status_code == 200:
        search_id=search_inst_resp.json()["id"]
        url = f'{conn.base_url}/api/metadataSearches/results?searchId={search_id}&offset=0&limit=-1' 
        search_resp = conn.get(url)
        if search_resp.status_code == 200:
            obj_d_l=search_resp.json()
            print(len(obj_d_l))
            return obj_d_l
        
def escape_filename(filename: str) -> str:

    filename = filename.strip().strip('.')

    return re.sub(r'[\\/:*?"<>|\r\n\t%&@€§$]', '_', filename)

redis_env_d={"mstr_dev":{
        "host":"redis-14995.c56.east-us.azure.redns.redis-cloud.com",
        "port":14995,
        "password":"KSvQA45EE8ZEJ3xcFvus96927CW56D8u",  
        "username":"default",  
        "decode_responses":True,
        "project_id":"B7CA92F04B9FAE8D941C3E9B7E0CD754",
        "redis_prefix":"mstr_dev"
},
        "mstr_test":{
        "host":"redis-13092.c282.east-us-mz.azure.redns.redis-cloud.com",
        "port":13092,
        "password":"pzSq7pQas9TFd6gMhcypYC70HRdyyvRv",
        "username":"default___",
        "decode_responses":True,
        "project_id":"00F3103E41FF3743A057C3B5313595B7",
        "redis_prefix":"mstr_test"
}}

redis_con_d=redis_env_d["mstr_dev"]
searches_used_in_prp_d_l=[{"search_id":"E6A7C8EE4BB767E8380E3A896092C0E6","run_fg":1},
                           {"search_id":"BA38E83548881BF83570CCABE2CB9B4B","run_fg":1}]

prefix_map ={'report_grid_and_graph': 'REPORT_DEFINITION',
 'prompt_double': 'PROMPT',
 'prompt_expression': 'PROMPT',
 'agg_metric': 'AGG_METRIC',
 'report_engine': 'REPORT_DEFINITION',
 'prompt_objects': 'PROMPT',
 'report_grid': 'REPORT_DEFINITION',
 'report_graph': 'REPORT_DEFINITION',
 'prompt_elements': 'PROMPT',
 'prompt_big_decimal': 'PROMPT',
 'prompt_string': 'PROMPT',
 'consolidation_element': 'CONSOLIDATION_ELEMENT',
 "14081": 'DOCUMENT_DEFINITION',
 'consolidation': 'CONSOLIDATION',
 'custom_group': 'FILTER',
 'prompt_date': 'PROMPT',
 'attribute':'ATTRIBUTE',
 'metric': 'METRIC',
 'filter': 'FILTER',
 'fact': 'FACT',
 'logical_table':'DBTABLE',
 'search':'SEARCH'}

## Connect to MSTR

In [2]:
with open('..\\config\\user_d.json', 'r') as openfile:
    user_d = json.load(openfile)
conn_params =  user_d["conn_params"]
conn = Connection(**conn_params)


Connection to Strategy One Intelligence Server has been established.
Connection to Strategy One Intelligence Server has been established.
No project selected.
Connection to Strategy One Intelligence Server has been established.
No project selected.


## Define read_out_obj_types

In [ ]:
folder_id="B2A671234A1B55E512EC3F83A24A150A"
project_id="B7CA92F04B9FAE8D941C3E9B7E0CD754"
conn.select_project(project_id)
all_obj_d_l=i_mstr_global.get_obj_from_sh_fold(conn,folder_id=folder_id)

In [ ]:
all_obj_d_l=[]
all_obj_d_l.append({"id":"5D55D4504907E73F560BFBA95AD22A66","type":"3","subtype":"768"})
#all_obj_d_l.append({"id":"98EB31B54122FFB738E6E08A2F29421A","type":"55","subtype":"14081"})
conn.select_project(project_id)


## Save objects to redis

In [3]:
# Initialize Redis connection
i_redis_bi_analysis = redis_bi_analysis( 
    host=redis_con_d["host"],
    port=redis_con_d["port"],
    password=redis_con_d["password"],
    username=redis_con_d["username"],
    decode_responses=redis_con_d["decode_responses"]
)

env_prefix="development"
i_redis_bi_analysis.emergency_flush_db(confirm_phrase='FLUSH_ALL_DATA')

for env in redis_env_d.keys():
    conn.select_project(redis_env_d[env]["project_id"])
    env_prefix=redis_env_d[env]["redis_prefix"]
    all_obj_d_l=read_out_project_all(conn)

    err_d_l=i_redis_mstr_json.save_obj_json_to_redis(i_redis_bi_analysis=i_redis_bi_analysis
                                                    ,conn=conn
                                                    ,prefix_map=prefix_map
                                                    , all_obj_d_l=all_obj_d_l
                                                    , env_prefix=env_prefix)

    i_redis_mstr_json.run_searches_redis(conn=conn
                                        ,i_redis_bi_analysis=i_redis_bi_analysis
                                        ,prefix_map=prefix_map
                                        ,env_prefix=env_prefix
                                        ,searches_used_in_prp_d_l=searches_used_in_prp_d_l)



JJJJJ
166
166
907
907
717
717
4
4
167
167
118
118
36
36
7
7
74
74
0
0
0
0
0
0
0
0
0
0
0
0
8
8
5
5
2
2
91
91
462
462
8
8
27
27
Uploaded object definition for 8FE9684E11D5287510006389EBC3E6A4
Uploaded object definition for 8FE9684E11D5287510006389EBC3E6A4
Uploaded object definition for 04373BB64B0177EA2A662E9AF58378F5
Uploaded object definition for 04373BB64B0177EA2A662E9AF58378F5
Uploaded object definition for 5EEF50EB42EEEBA8CBCC20B42FC5162D
Uploaded object definition for 5EEF50EB42EEEBA8CBCC20B42FC5162D
Uploaded object definition for 613437284BB4A2849B52F4A31D80D418
Uploaded object definition for 613437284BB4A2849B52F4A31D80D418
Uploaded object definition for C66FB1D911D3EB0CC000B4B2D86C964F
Uploaded object definition for C66FB1D911D3EB0CC000B4B2D86C964F
Uploaded object definition for 085CBF314703A71E926EA187C2002969
Uploaded object definition for 8045236C492EB6A00E795996F0302F18
Uploaded object definition for 085CBF314703A71E926EA187C2002969
Uploaded object definition for 8045236C492